# Books based dataset

Let's align the Russian classics literature.

In [187]:
import re
import numpy as np
import pickle
import pandas as pd
from bs4 import BeautifulSoup
import urllib
import codecs
import os,glob
import glob
import re
import urllib.request

from matplotlib import pyplot as plt

import razdel
import time

## Островский. "Как закалялась сталь".

In [96]:
text_ru = "D:\\ml\\books\\stal_ru_1.txt"
text_zh = "D:\\ml\\books\\stal_zh_1.txt"
out_ru = "D:\\ml\\books\\wm_zh.txt"
out_zh = "D:\\ml\\books\\wm_ru.txt"

with open(text_ru, mode='r', encoding='utf-8') as input_ru, open(text_zh, mode='r', encoding='utf-8') as input_zh, \
        open(out_ru, mode='w', encoding='utf-8') as out_ru, open(out_zh, mode='w', encoding='utf-8') as out_zh:
    ru = ' '.join(input_ru.readlines())
    zh = ''.join(input_zh.readlines())

def split_zh(paragraph):
    for sent in re.findall(u'[^!?。！？\.\!\?]+[!?。！？\.\!\?]?', paragraph, flags=re.U):
        yield sent
        
sent_ru = list(x.text for x in razdel.sentenize(ru))
sent_zh = list(split_zh(zh))

In [97]:
len(list(sent_ru))

387

In [98]:
len(list(sent_zh))

457

In [99]:
for ru,zh in zip(list(sent_ru),list(sent_zh)):
    print(ru)
    print(zh)
    print()

- Кто  из вас перед праздником приходил  ко мне  домой отвечать урок  -
 встаньте!
  “节前上我家去补考的，都给我站起来！

Обрюзглый человек в рясе,  с тяжелым крестом на шее угрожающе посмотрел
 на учеников.
”
    一个脸皮松弛的胖神甫，身上穿着法衣，脖子上挂着沉甸甸的十字架，气势汹汹地
瞪着全班的学生。

Дети боязливо посматривали на
 человека в рясе.
孩子们惊恐不安地望着他。

-  Вы садитесь.

    “你们俩坐下。

Махнул поп  в  сторону  девочек.
”神甫朝女孩子挥挥手说。

Те  быстро  сели,
 облегченно вздохнув.

    她们急忙坐下，松了一口气。

Глазки отца Василия сосредоточились на четырех фигурках.

    瓦西里神甫那对小眼睛死盯在四个男孩子身上。

- Идите-ка сюда, голубчики!

    “过来吧，宝贝们！

Отец  Василий поднялся, отодвинул стул и подошел вплотную к сбившимся в
 кучу ребятам.
”
    瓦西里神甫站起来，推开椅子，走到挤作一团的四个孩子跟前。

- Кто из вас, подлецов, курит?

    “你们这几个小无赖，谁抽烟？

Все четверо тихо ответили:
      - Мы не курим, батюшка.
”
    四个孩子都小声回答：“我们不会抽，神甫。

Лицо попа побагровело.
”
    神甫脸都气红了。

- Не курите, мерзавцы, а, махорку кто в тесто насыпал?

    “混帐东西，不会抽，那发面里的烟末是谁撒的？

Не курите?
都不会抽吗？

А вот
 мы  сейчас  посмотр

Слишком мног несоответствий. Тексты быстро разъезжаются, нужны специальные техники.

## Rucorpora

In [199]:
sent_ru = []
sent_zh = []

def extract_sentences(url, mode):
    fp = urllib.request.urlopen(url)
    mybytes = fp.read()
    fp.close()
    
    html=BeautifulSoup(mybytes)

    for x in html.find_all("table", {"class":"para"}):    
        i=0
        for y in x.find_all("td", {"class":"para-snippet"}):
            li = y.find("li")

            #clean
            for t in li.find_all("span",{"class":"doc"}):
                t.decompose()        
            for t in li.find_all("span",{"class":"off"}):
                t.decompose()            
            for t in li.find_all("a"):
                t.decompose()        

            for z in li.find_all("span",{"class":"b-wrd-expl"}):
                z.replace_with(z.string)

            i+=1
            if mode == "ru":
                if i == 1:
                    sent_ru.append(li.text.strip())
                elif i == 2:
                    sent_zh.append(li.text.strip().replace(" ",""))
                else:
                    continue
            elif mode == "cn":
                if i == 1:
                    sent_zh.append(li.text.strip().replace(" ",""))
                elif i == 3:
                    sent_ru.append(li.text.strip())

#             print(li.text)
    
    return html

### "Как закалялась сталь"

In [189]:
url = "http://search1.ruscorpora.ru/search.xml?mycorp=(lang%3A%22zho%22%20%7C%20lang_trans%3A%22zho%22)&mysent=&mysize=279478&mysentsize=15735&dpp=&spp=&spd=&text=lexform&mode=para&sort=gr_tagging&env=alpha&req=*&docid=1235&ps="

In [190]:
for i in range(24):
    print("page", i)
    url_i = url+str(i)
    extract_sentences(url_i)
    time.sleep(5)    

  – Кто из вас перед праздником приходил ко мне домой отвечать урок – встаньте!     
 节 前 上 我 家 去 补考 的 “都 给 我 站起来，     
 Обрюзглый человек в рясе, с тяжелым крестом на шее, угрожающе посмотрел на учеников.     
 一 脖子 上 挂 着 沉甸甸 的 十字架， 气势汹汹 地 瞪 着 全班 的 学生。     
 Маленькие злые глазки точно прокалывали всех шестерых, поднявшихся со скамеек, – четырех мальчиков и двух девочек.     
 六 个 学生 应声 从 板凳 上 站 了 起来， 四 个 男生， 两 个 女生。 神甫 两 只 小 眼睛 闪 着 凶光， 像 要 把 他们 一口 吞 下去 似的。     
 Дети боязливо посматривали на человека в рясе.     
 孩子们 惊恐 不安 地 望 着 他。     
  – Вы садитесь, – махнул поп в сторону девочек.     
 你们 俩 坐下 “神甫 朝 女孩子 挥 挥手 说。     
  – Вы садитесь, – махнул поп в сторону девочек.    Те быстро сели, облегченно вздохнув.    Глазки отца Василия сосредоточились на четырех фигурках.     
 你们 俩 坐下 “神甫 朝 女孩子 挥 挥手 说。    她们 急忙 坐下， 松 了 一口气。    瓦西里 神甫 那 对 小 眼睛 死 盯 在 四 个 男孩子 身上。     
 Глазки отца Василия сосредоточились на четырех фигурках.     
 瓦西里 神甫 那 对 小 眼睛 死 盯 在 四 个 男孩子 身上。     
 Глазки отца Василия

 Все тропари, Новый и Ветхий завет знал он на зубок; твердо знал, в какой день что произведено богом.     
 新约、 旧约 和 所有 的 祈祷 词， 他 都 背 得 滚瓜烂熟。 上帝 哪 一 天 创造 了 什么， 他 也 都 记得 一清二楚。     
 Все тропари, Новый и Ветхий завет знал он на зубок; твердо знал, в какой день что произведено богом.    Павка решил расспросить отца Василия.    На первом же уроке закона, едва поп уселся в кресло, Павка поднял руку и, получив разрешение говорить, встал:     
 新约、 旧约 和 所有 的 祈祷 词， 他 都 背 得 滚瓜烂熟。 上帝 哪 一 天 创造 了 什么， 他 也 都 记得 一清二楚。    保尔 打 定 主意， 要 向 瓦西里 神甫 问 个 明白。    等到 上 圣经 课 的 时候， 神甫 刚 坐 到 椅子 上， 保尔 就 举起 手 来， 得到 允许 以后， 他 站起来 说：     
 На первом же уроке закона, едва поп уселся в кресло, Павка поднял руку и, получив разрешение говорить, встал:     
 等到 上 圣经 课 的 时候， 神甫 刚 坐 到 椅子 上， 保尔 就 举起 手 来， 得到 允许 以后， 他 站起来 说：     
  – Батюшка, а почему учитель в старшем классе говорит, что земля миллион лет стоит, а не как в законе божием – пять тыс. – и сразу осел от визгливого крика отца Василия:     
 神甫 “为什么 高年 级 老师 说， 地球 已经 

  «Тетка эта, видно, ничего», – решил он про себя и, осмелев, обратился к Фросе:    – А что мне сейчас делать, тетя?    Сказал и запнулся. Громкий хохот работавших в судомойне женщин покрыл его последние слова:     
 看样子 这位 大 婶 还 不错 “他 心里 这样 想。 便 鼓起 勇气 问 弗 罗 霞”    那 我 现在 干 些 什么 呢 “大 婶，    他 说 到 这里， 洗刷 间 的 女工 们 一阵 哈哈大笑， 淹没 了 他 的话， 他 愣 住 了。     
 Сказал и запнулся. Громкий хохот работавших в судомойне женщин покрыл его последние слова:     
 他 说 到 这里， 洗刷 间 的 女工 们 一阵 哈哈大笑， 淹没 了 他 的话， 他 愣 住 了。     
 Сказал и запнулся. Громкий хохот работавших в судомойне женщин покрыл его последние слова:    – Ха-ха-ха!.    У Фросеньки уж и племянник завелся.     
 他 说 到 这里， 洗刷 间 的 女工 们 一阵 哈哈大笑， 淹没 了 他 的话， 他 愣 住 了。    哈哈 哈 “    弗 罗 霞 这 回 捡 了 笑 得 比 谁 都 厉害！     
  – Ха-ха-ха!.    У Фросеньки уж и племянник завелся.    Павка из-за пара не разглядел ее лица, а Фросе всего было восемнадцать лет.     
 哈哈 哈 “    弗 罗 霞 这 回 捡 了 笑 得 比 谁 都 厉害！    因为 屋 里 全 是 蒸汽， 保尔 没有 看清 弗 罗 霞 的 脸， 其实 她 只有 十八 岁。     
 Павка из-за пар

 Нахальный тон и вызывающее поведение мальчишки разозлили Павку.     
 那个 男孩 的 无赖 腔调 和 挑衅 态度 激怒 了 保尔。     
 Он подвинулся на шаг к своей смене, приготовясь влепить мальчишке хорошего леща, но боязнь быть прогнанным в первый же день работы остановила его.     
 他 朝 男孩 逼近 一 步， 本来 想 狠狠 揍 他 一 顿， 但是 又 怕 头一 天上 工 就 给 开除， 才 忍 住 了。     
 Он подвинулся на шаг к своей смене, приготовясь влепить мальчишке хорошего леща, но боязнь быть прогнанным в первый же день работы остановила его.    Весь потемнев, он сказал:    – Ты потише, не налетай, а то обожжешься.     
 他 朝 男孩 逼近 一 步， 本来 想 狠狠 揍 他 一 顿， 但是 又 怕 头一 天上 工 就 给 开除， 才 忍 住 了。    他 铁青 着 脸 说：    你 老实 点 “别 吓唬 人， 搬 起 石头 砸 自己 脚，     
  – Ты потише, не налетай, а то обожжешься.     
 你 老实 点 “别 吓唬 人， 搬 起 石头 砸 自己 脚，     
 Завтра приду в семь, а драться я умею не хуже тебя; если захочешь попробовать – пожалуйста.     
 明天 我 就 七 点 来， 要说 打架， 我 可不 在乎 你， 你 想 试 试， 那 就 请 吧！     
 Противник отодвинулся на шаг к кубу и с удивлением смотрел на взъерошенного Павку. 

  – Я пойду по делу на часок.    – И, согнувшись у притолоки двери, вышел.    Уже во дворе, проходя мимо окна, сказал:     
 我 出去 个把 钟头 “办 点 事，    说 完， 一 弯腰， 跨 出 了 房门。    他 走 到 院子 里， 从 窗 前 经过 的 时候， 又 说：     
 Уже во дворе, проходя мимо окна, сказал:     
 他 走 到 院子 里， 从 窗 前 经过 的 时候， 又 说：     
  – Там тебе привез сапоги и ножик, мамка даст.     
 我 给 你 带来 一 双 靴子 和 一 把 小刀 “妈 会 拿 给 你 的，     
  – Там тебе привез сапоги и ножик, мамка даст.    Буфет вокзала торговал беспрерывно целые сутки.    Железнодорожный узел соединял пять линий.     
 我 给 你 带来 一 双 靴子 和 一 把 小刀 “妈 会 拿 给 你 的，    车站 食堂 昼夜 不停 地 营业。    有 六 铁路 通 到 这个 枢纽 站。     
 Буфет вокзала торговал беспрерывно целые сутки.    Железнодорожный узел соединял пять линий.    Вокзал плотно был набит людьми и только на два-три часа ночью, в перерыв между двумя поездами, затихал.     
 车站 食堂 昼夜 不停 地 营业。    有 六 铁路 通 到 这个 枢纽 站。    车站 总是 挤满 了 人， 只有 夜里， 在 两 班 火车 的 间隙， 才能 安静 两 三 钟头。     
 Вокзал плотно был набит людьми и только на два-три часа ночью, в

 Сверху по лестнице еще кто-то сбегал поспешными, легкими шагами, и Павка услыхал знакомый голос:     
 正在 这时 候， 又 有人 从 上面 轻轻 地 快步跑 下来， 保尔 听到 了 一 声音：     
 Сверху по лестнице еще кто-то сбегал поспешными, легкими шагами, и Павка услыхал знакомый голос:    – Прохошка, подожди.    Прохошка остановился и, обернувшись, посмотрел вверх.     
 正在 这时 候， 又 有人 从 上面 轻轻 地 快步跑 下来， 保尔 听到 了 一 声音：    普罗 霍尔 “你等 一下，    普罗 霍尔 站住 了， 掉头 朝 上面 看 了 一眼。     
  – Прохошка, подожди.    Прохошка остановился и, обернувшись, посмотрел вверх.    – Тебе чего? – буркнул он.     
 普罗 霍尔 “你等 一下，    普罗 霍尔 站住 了， 掉头 朝 上面 看 了 一眼。    什么事 “他 咕哝 了 一句？     
 Прохошка остановился и, обернувшись, посмотрел вверх.    – Тебе чего? – буркнул он.    Шаги на лестнице застучали вниз, и Павка узнал Фросю.     
 普罗 霍尔 站住 了， 掉头 朝 上面 看 了 一眼。    什么事 “他 咕哝 了 一句？    有人 顺着 楼梯 走 了 下来， 保尔 认出 是 弗 罗 霞。     
 Шаги на лестнице застучали вниз, и Павка узнал Фросю.     
 有人 顺着 楼梯 走 了 下来， 保尔 认出 是 弗 罗 霞。     
 Она взяла официанта за рукав и прерывающим

  – И руки его, лежавшие на коленях, сжались в кулаки.     
 他 把 放 在 膝 上 的 两 只 手 攥 成了 拳头。     
  – Что это на тебя сегодня нашло? – продолжал приподнявшийся на локтях Климка.     
 你 今天 是 怎么了 “克 利 姆 卡 用 胳膊 支 起身 子？ 接着 问”     
  – Что это на тебя сегодня нашло? – продолжал приподнявшийся на локтях Климка.    – Сегодня нашло, говоришь?    Всегда находило, как только попал сюда работать.     
 你 今天 是 怎么了 “克 利 姆 卡 用 胳膊 支 起身 子？ 接着 问”    你 问 我 今天 怎么了 “    我 从 到 这儿 来 干活 的 那天 起， 就 一直 不怎么 的。     
 Всегда находило, как только попал сюда работать.     
 我 从 到 这儿 来 干活 的 那天 起， 就 一直 不怎么 的。     
 Всегда находило, как только попал сюда работать.    Ты погляди, что здесь делается!    Работаем, как верблюды, а в благодарность тебя по зубам бьет кто только вздумает, и ни от кого защиты нет.     
 我 从 到 这儿 来 干活 的 那天 起， 就 一直 不怎么 的。    你 看看， 这儿 是 个 什么地方！    咱们 像 骆驼 一样 干活， 可得到 的 报答 呢， 是 谁 高兴 谁 就 赏 你 几个 嘴巴子， 连 一 个 护着 你 的 人 都 没有。     
 Работаем, как верблюды, а в благодарность тебя по зубам бьет кто только взду

 Ручейки подбирались под вещи и чемоданы спящих пассажиров. Никто этого не замечал, и, только когда вода залила лежавшего на полу пассажира и тот, вскочив на ноги, закричал, все бросились к вещам.     
 一 股 股 水流 悄悄 地 流 到 熟睡 的 旅客 们 的 行李 下面， 谁 也 没有 发觉。 直到 水 浸 醒 了 一 个 躺 在地 板 上 的 旅客， 他 一下 跳 起来， 大喊大叫， 其他 旅客 才 慌忙 去 抢 自己 的 行李。     
 Ручейки подбирались под вещи и чемоданы спящих пассажиров. Никто этого не замечал, и, только когда вода залила лежавшего на полу пассажира и тот, вскочив на ноги, закричал, все бросились к вещам.    Поднялась суматоха.    Крики усилились.     
 一 股 股 水流 悄悄 地 流 到 熟睡 的 旅客 们 的 行李 下面， 谁 也 没有 发觉。 直到 水 浸 醒 了 一 个 躺 在地 板 上 的 旅客， 他 一下 跳 起来， 大喊大叫， 其他 旅客 才 慌忙 去 抢 自己 的 行李。    食堂 里 收拾 桌子 的 普罗 尔 听到 旅客 的 喊叫 声， 急忙 跑 过来。 他 跳 过 积水， 冲 到 门 旁， 用力 把门 打开， 原来 被 门挡 住 的 水 一下子 全 涌进 了 餐室。    喊叫 声 更 大 了。     
 Поднялась суматоха.    Крики усилились.    В судомойню вбежали дежурные официанты.     
 食堂 里 收拾 桌子 的 普罗 尔 听到 旅客 的 喊叫 声， 急忙 跑 过来。 他 跳 过 积水， 冲 到 门 旁， 用力 把门 打开， 原来 被 门挡 住 的 水 一下子 全 涌进 了 餐

In [192]:
out_ru = "D:\\ml\\books\\stal_ru.txt"
out_zh = "D:\\ml\\books\\stal_zh.txt"

with open(out_ru, mode='w', encoding='utf-8') as output_ru, open(out_zh, mode='w', encoding='utf-8') as output_zh:
    for ru, zh in zip(sent_ru, sent_zh):
        output_ru.write(ru + '\n')  
        output_zh.write(zh + '\n')    


### Get the rest

In [196]:
path = "http://search1.ruscorpora.ru/search.xml?text=lexform&mode=para&req=*&docid="
    
urls = [("ben_ye","1916",13,"cn"),
    ("zhufu","1915",17,"cn"),
    ("kuangren","1917",13,"cn"),
    ("ostr_stal","1235",23,"ru"),
    ("chelovek","1231",15,"ru"),
    ("izergil","1234",4,"ru"),
    ("tolst","1232",5,"ru"),
    ("idiot","1233",142,"ru"),
    ("lubov","1236",82,"ru"),
    ("mumu","1237",10,"ru")
]

for url in urls:
    print("url:", path + url[1] + "&ps=", "pages:", url[2])

url: http://search1.ruscorpora.ru/search.xml?text=lexform&mode=para&req=*&docid=1916&ps= pages: 13
url: http://search1.ruscorpora.ru/search.xml?text=lexform&mode=para&req=*&docid=1915&ps= pages: 17
url: http://search1.ruscorpora.ru/search.xml?text=lexform&mode=para&req=*&docid=1917&ps= pages: 13
url: http://search1.ruscorpora.ru/search.xml?text=lexform&mode=para&req=*&docid=1235&ps= pages: 23
url: http://search1.ruscorpora.ru/search.xml?text=lexform&mode=para&req=*&docid=1231&ps= pages: 15
url: http://search1.ruscorpora.ru/search.xml?text=lexform&mode=para&req=*&docid=1234&ps= pages: 4
url: http://search1.ruscorpora.ru/search.xml?text=lexform&mode=para&req=*&docid=1232&ps= pages: 5
url: http://search1.ruscorpora.ru/search.xml?text=lexform&mode=para&req=*&docid=1233&ps= pages: 142
url: http://search1.ruscorpora.ru/search.xml?text=lexform&mode=para&req=*&docid=1236&ps= pages: 82
url: http://search1.ruscorpora.ru/search.xml?text=lexform&mode=para&req=*&docid=1237&ps= pages: 10


In [204]:
out = "D:\\ml\\books\\"
out_ru_total = out + "books_ru.txt"
out_zh_total = out + "books_zh.txt"

with open(out_ru_total, mode='w', encoding='utf-8') as output_ru_total, open(out_zh_total, mode='w', encoding='utf-8') as output_zh_total:
    for url in urls:
        sent_ru = []
        sent_zh = []
        book_url = path + url[1] + "&ps="
        print("url:", book_url, "pages:", url[2], "mode:", url[3])

        for i in range(url[2]):
            print("page", i)
            url_i = book_url+str(i)
            extract_sentences(url_i, url[3])
            time.sleep(2)

        out_ru = out + url[0] + "_ru.txt"
        out_zh = out + url[0] + "_zh.txt"
            
        with open(out_ru, mode='w', encoding='utf-8') as output_ru, open(out_zh, mode='w', encoding='utf-8') as output_zh:
            for ru, zh in zip(sent_ru, sent_zh):
                output_ru.write(ru + '\n')  
                output_zh.write(zh + '\n')   
                output_ru_total.write(ru + '\n')  
                output_zh_total.write(zh + '\n')   



url: http://search1.ruscorpora.ru/search.xml?text=lexform&mode=para&req=*&docid=1916&ps= pages: 13 mode: cn
page 0
page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10
page 11
page 12
url: http://search1.ruscorpora.ru/search.xml?text=lexform&mode=para&req=*&docid=1915&ps= pages: 17 mode: cn
page 0
page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10
page 11
page 12
page 13
page 14
page 15
page 16
url: http://search1.ruscorpora.ru/search.xml?text=lexform&mode=para&req=*&docid=1917&ps= pages: 13 mode: cn
page 0
page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10
page 11
page 12
url: http://search1.ruscorpora.ru/search.xml?text=lexform&mode=para&req=*&docid=1235&ps= pages: 23 mode: ru
page 0
page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10
page 11
page 12
page 13
page 14
page 15
page 16
page 17
page 18
page 19
page 20
page 21
page 22
url: http://search1.ruscorpora.ru/search.xml?text=lexform&mode=para&req=*&docid=